## Library Imports

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import requests
import xlsxwriter
import math

## Stock Imports

In [ ]:
symbols = pd.read_csv('../data/sp500_symbols.csv', skiprows=1, header=None)[0].tolist()

## Download Historical Data

In [ ]:
all_data = {}
for symbol in symbols:
    try:
        data = yf.download(symbol, period="1y")['Adj Close']
        if not data.empty:
            all_data[symbol] = data
    except Exception as e:
        print(f"Failed download for {symbol}: {e}")

all_data_df = pd.DataFrame(all_data)  # Convert the dictionary to a DataFrame
all_data_df = all_data_df.dropna(axis=1, how='all')  # Drop columns with missing values

## Calculate Equal Weights For Each Stock

In [ ]:
num_stocks = all_data_df.shape[1]
equal_weights = 1 / num_stocks

## Calculate Daily Returns For Each Stock

In [ ]:
daily_returns = all_data_df.pct_change().dropna()

## Fetching Stock Data using yfinance

In [ ]:
def get_stock_info(symbol):
    stock = yf.Ticker(symbol)
    info = stock.info
    market_cap = info.get('marketCap', 'N/A')
    price = info.get('currentPrice', 'N/A')
    return market_cap, price

data_list = [] # List to collect data

valid_symbols = list(all_data.keys()) # Fetch data for each valid stock
for symbol in valid_symbols:
    try:
        market_cap, price = get_stock_info(symbol)
        data_list.append({'Ticker': symbol, 'Price': price, 'Market Capitalization': market_cap, 'Number Of Shares to Buy': 'N/A'})
    except Exception as e:
        print(f"Failed to get info for {symbol}: {e}")

final_dataframe = pd.DataFrame(data_list)
print(final_dataframe)

## Calculate Portfolio Returns

In [ ]:
weighted_returns = daily_returns * equal_weights
portfolio_returns = weighted_returns.sum(axis=1)
cumulative_returns = (1 + portfolio_returns).cumprod()

## Plot Cumulative Returns

In [ ]:
plt.figure(figsize=(10, 6))
cumulative_returns.plot()
plt.title('Equal-Weighted S&P 500 Portfolio Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.show()

## Input for Portfolio Size

In [ ]:
while True:
    portfolio_size = input("Enter the value of your portfolio:")
    try:
        val = float(portfolio_size)
        break
    except ValueError:
        print("That's not a number! \nTry again:")

## Calculate Sharpe Ratio

In [ ]:
risk_free_rate = 0.01  # Example risk-free rate
excess_returns = portfolio_returns - (risk_free_rate / 252)
sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(252)
print(f"Sharpe Ratio: {sharpe_ratio}")

## Calculate Drawdown

In [ ]:
rolling_max = cumulative_returns.cummax()
drawdown = (cumulative_returns - rolling_max) / rolling_max
max_drawdown = drawdown.min()
print(f"Max Drawdown: {max_drawdown}")

## Input for Portfolio Size

In [ ]:
position_size = val / len(final_dataframe.index)
for i in range(len(final_dataframe)):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
# Filter out stocks which are not bought
final_dataframe = final_dataframe[final_dataframe['Number Of Shares to Buy'] > 0]

In [ ]:
print(final_dataframe)

## Create Final DataFrame

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index=False)

## Create and Apply Formats

In [ ]:
background_color = '#ffffff'
font_color = '#000000'

string_format = writer.book.add_format(
    {'font_color': font_color, 'bg_color': background_color, 'border': 1}
)

dollar_format = writer.book.add_format(
    {'num_format': '$0.00', 'font_color': font_color, 'bg_color': background_color, 'border': 1}
)

integer_format = writer.book.add_format(
    {'num_format': '0', 'font_color': font_color, 'bg_color': background_color, 'border': 1}
)

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number Of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Save Excel Output

In [ ]:
writer.close()